In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import scipy as sp
import string
import json
import re
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as  plt
import seaborn as sns
import sklearn
from sklearn.metrics import accuracy_score
####
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression
import sklearn.linear_model as linear_model
import sklearn.model_selection as model_selection

from sklearn import svm
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB

In [2]:
# open training dataset and parse recipes

with open('train.json') as cooking_file:  
    
    data = json.load(cooking_file)
    

In [3]:
#removing punctuations and spaces before fixing the data and keep them all in a dict  (italian, mexican)

punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

def gather_recipes(recipes, cuisine_country):
    
    data_list = []
    
    for _object in recipes:
        
        if _object.get('cuisine') == cuisine_country:
            
            ingredients = _object.get('ingredients') 
            _id = _object.get('id')
            
            for i in ingredients:
                
                _dict = dict()
                ingr = i.replace(' ', '').lower()
                    
                for ch in ingr: 
                    
                    if ch in punctuations: 
                        
                        ingr = ingr.replace(ch, '')  
                
                _dict['ingredients'] = ingr                
                _dict['id'] = _id
                data_list.append(_dict)
                
    return data_list

clean_italian_cuisine_data = gather_recipes(data, 'italian')
clean_mexican_cuisine_data = gather_recipes(data, 'mexican')

In [4]:
#creating dfs  (italian, mexican)

italian_df = DataFrame(clean_italian_cuisine_data)
italian_ingredients = list(set(italian_df.ingredients))

mexican_df = DataFrame(clean_mexican_cuisine_data)
mexican_ingredients = list(set(mexican_df.ingredients))


In [5]:
#fixing data for CountVectorizer (italian_df)

it_unique_ids = []
it_ingredients = []
it_grouped = italian_df.groupby('id')

for ids, ing in it_grouped:
    
    it_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    it_ingredients.append(row)
    

In [6]:
#fixing data for CountVectorizer (mexican_df)

mex_unique_ids = []
mex_ingredients = []
mex_grouped = mexican_df.groupby('id')

for ids, ing in mex_grouped:
    
    mex_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    mex_ingredients.append(row)
    


In [7]:
#using CountVectorizer  (italian, mexican) and adding one column Cuisine

it_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
it_array = it_vectorizer.fit_transform(it_ingredients)
it_array = it_array.toarray()

it_df = DataFrame(it_array, columns = it_vectorizer.get_feature_names(), index = it_unique_ids)
it_df['Cuisine'] = 0 #italian
it_df.to_csv('it_cleandata.csv', index_label = 'ID')
print(it_df)

mex_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
mex_array = mex_vectorizer.fit_transform(mex_ingredients)
mex_array = mex_array.toarray()

mex_df = DataFrame(mex_array, columns = mex_vectorizer.get_feature_names(), index = mex_unique_ids)
mex_df['Cuisine'] = 1 #mexican
mex_df.to_csv('mex_cleandata.csv', index_label = 'ID')
print(mex_df)

       abbamele  accent  acinipepe  acornsquash  activedryyeast  adobosauce  \
4             0       0          0            0               0           0   
14            0       0          0            0               0           0   
20            0       0          0            0               0           0   
56            0       0          0            0               0           0   
62            0       0          0            0               0           0   
...         ...     ...        ...          ...             ...         ...   
49679         0       0          0            0               0           0   
49681         0       0          0            0               0           0   
49689         0       0          0            0               0           0   
49697         0       0          0            0               0           0   
49708         0       0          0            0               0           0   

       adoboseasoning  agavenectar  agedbalsamicvin

In [8]:
#putting mexican and italian in one DataFrame

df = pd.DataFrame(it_df)
df = df.append(mex_df, sort = False)
df.fillna(0, inplace = True)
df = df.astype('int64')

In [9]:
X = df.loc[:, df.columns != 'Cuisine'].values  #all columns with values except Cuisine column
print(X.shape)

Y = df['Cuisine'] #just the ids in a list

(14276, 3976)


In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 0)

In [11]:
# Multiple Logistic Regression 

lr_m = linear_model.LogisticRegression()
lr_m.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
print("classifier score:",lr_m.score(X_test, Y_test))
Y_pred = lr_m.predict(X_test)
print("\naccuracy:",metrics.accuracy_score(Y_test, Y_pred))
print("\nConfusion matrix:")
print(metrics.confusion_matrix(Y_test, Y_pred))
print("\nPrecision Score per class:")
print(metrics.precision_score(Y_test, Y_pred, average = None))
print("\nAverage Precision Score:")
print(metrics.precision_score(Y_test, Y_pred, average = 'weighted'))
print("\nRecall Score per class:")
print(metrics.recall_score(Y_test, Y_pred, average = None))
print("\nAverage Recall Score:")
print(metrics.recall_score(Y_test, Y_pred, average = 'weighted'))
print("\nF1-score Score per class:")
print(metrics.f1_score(Y_test, Y_pred, average = None))
print("\nAverage F1 Score:")
print(metrics.f1_score(Y_test, Y_pred, average = 'weighted'))

report = metrics.classification_report(Y_test,Y_pred)
print(report)

classifier score: 0.9686186606892687

accuracy: 0.9686186606892687

Confusion matrix:
[[1887   40]
 [  72 1570]]

Precision Score per class:
[0.96324655 0.97515528]

Average Precision Score:
0.9687254354733641

Recall Score per class:
[0.97924235 0.95615104]

Average Recall Score:
0.9686186606892687

F1-score Score per class:
[0.97117859 0.96555966]

Average F1 Score:
0.9685934707348253
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1927
           1       0.98      0.96      0.97      1642

    accuracy                           0.97      3569
   macro avg       0.97      0.97      0.97      3569
weighted avg       0.97      0.97      0.97      3569



In [13]:
scores = model_selection.cross_val_score(lr_m, X, Y, scoring = 'f1_weighted', cv = 5)
print('scores:', scores)
print('mean scores:', scores.mean())

scores: [0.96809087 0.97195581 0.96879183 0.96668815 0.96880957]
mean scores: 0.9688672451742961


In [14]:
# Support Vector Machines

svm_m = svm.SVC()
svm_m.fit(X_train, Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [15]:
print("classifier score:",svm_m.score(X_test, Y_test))
Y_pred = svm_m.predict(X_test)
print("\naccuracy:",metrics.accuracy_score(Y_test, Y_pred))
print("\nConfusion matrix:")
print(metrics.confusion_matrix(Y_test, Y_pred))
print("\nPrecision Score per class:")
print(metrics.precision_score(Y_test, Y_pred, average = None))
print("\nAverage Precision Score:")
print(metrics.precision_score(Y_test, Y_pred, average = 'weighted'))
print("\nRecall Score per class:")
print(metrics.recall_score(Y_test, Y_pred, average= None))
print("\nAverage Recall Score:")
print(metrics.recall_score(Y_test, Y_pred, average = 'weighted'))
print("\nF1-score Score per class:")
print(metrics.f1_score(Y_test, Y_pred, average = None))
print("\nAverage F1 Score:")
print(metrics.f1_score(Y_test, Y_pred, average = 'weighted'))

report = metrics.classification_report(Y_test,Y_pred)
print(report)

classifier score: 0.9649761838049874

accuracy: 0.9649761838049874

Confusion matrix:
[[1894   33]
 [  92 1550]]

Precision Score per class:
[0.95367573 0.97915351]

Average Precision Score:
0.9653973630645947

Recall Score per class:
[0.98287494 0.94397077]

Average Recall Score:
0.9649761838049874

F1-score Score per class:
[0.9680552  0.96124031]

Average F1 Score:
0.9649198545052377
              precision    recall  f1-score   support

           0       0.95      0.98      0.97      1927
           1       0.98      0.94      0.96      1642

    accuracy                           0.96      3569
   macro avg       0.97      0.96      0.96      3569
weighted avg       0.97      0.96      0.96      3569



In [16]:
#Decision Trees

dtree = tree.DecisionTreeClassifier()
dtree.fit(X_train, Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [17]:
print("classifier score:",dtree.score(X_test, Y_test))
Y_pred = dtree.predict(X_test)
Y_prob = dtree.predict_proba(X_test)
print('Y probability:', Y_prob)

print("classifier predictions:", Y_pred)
print("ground truth labels   :", Y_test)

print("accuracy:", metrics.accuracy_score(Y_test, Y_pred))
print("\nConfusion matrix:")
print(metrics.confusion_matrix(Y_test, Y_pred))
print("\nPrecision Score per class:")
print(metrics.precision_score(Y_test, Y_pred, average=None))
print("\nAverage Precision Score:")
print(metrics.precision_score(Y_test, Y_pred, average='weighted'))
print("\nRecall Score per class:")
print(metrics.recall_score(Y_test, Y_pred, average=None))
print("\nAverage Recall Score:")
print(metrics.recall_score(Y_test, Y_pred, average='weighted'))
print("\nF1-score Score per class:")
print(metrics.f1_score(Y_test, Y_pred, average=None))
print("\nAverage F1 Score:")
print(metrics.f1_score(Y_test, Y_pred, average='weighted'))

report = metrics.classification_report(Y_test,Y_pred)
print(report)

classifier score: 0.9352759876716167
Y probability: [[0. 1.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]
classifier predictions: [1 0 0 ... 1 0 1]
ground truth labels   : 41913    1
39513    0
31187    0
47868    1
26047    0
        ..
21426    0
16747    0
37271    1
13967    0
7027     1
Name: Cuisine, Length: 3569, dtype: int64
accuracy: 0.9352759876716167

Confusion matrix:
[[1810  117]
 [ 114 1528]]

Precision Score per class:
[0.94074844 0.92887538]

Average Precision Score:
0.9352859678292036

Recall Score per class:
[0.93928386 0.93057247]

Average Recall Score:
0.9352759876716167

F1-score Score per class:
[0.94001558 0.92972315]

Average F1 Score:
0.935280313433129
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      1927
           1       0.93      0.93      0.93      1642

    accuracy                           0.94      3569
   macro avg       0.93      0.93      0.93      3569
weighted avg       0.94      0.94   

In [18]:
#k-NN

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train,Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [19]:
print("classifier score:", knn.score(X_test, Y_test))
Y_pred = knn.predict(X_test)
print("\naccuracy:",metrics.accuracy_score(Y_test, Y_pred))
print("\nConfusion matrix:")
print(metrics.confusion_matrix(Y_test, Y_pred))
print("\nPrecision Score per class:")
print(metrics.precision_score(Y_test, Y_pred, average = None))
print("\nAverage Precision Score:")
print(metrics.precision_score(Y_test, Y_pred, average = 'weighted'))
print("\nRecall Score per class:")
print(metrics.recall_score(Y_test, Y_pred, average = None))
print("\nAverage Recall Score:")
print(metrics.recall_score(Y_test, Y_pred, average = 'weighted'))
print("\nF1-score Score per class:")
print(metrics.f1_score(Y_test, Y_pred, average = None))
print("\nAverage F1 Score:")
print(metrics.f1_score(Y_test, Y_pred, average = 'weighted'))

report = metrics.classification_report(Y_test,Y_pred)
print('Report:', report)

classifier score: 0.8809190249369572

accuracy: 0.8809190249369572

Confusion matrix:
[[1778  149]
 [ 276 1366]]

Precision Score per class:
[0.86562804 0.90165017]

Average Precision Score:
0.8822008432378832

Recall Score per class:
[0.92267774 0.8319123 ]

Average Recall Score:
0.8809190249369572

F1-score Score per class:
[0.8932429  0.86537852]

Average F1 Score:
0.8804232591419874
Report:               precision    recall  f1-score   support

           0       0.87      0.92      0.89      1927
           1       0.90      0.83      0.87      1642

    accuracy                           0.88      3569
   macro avg       0.88      0.88      0.88      3569
weighted avg       0.88      0.88      0.88      3569



In [20]:
# Naive Bayes

gnb = BernoulliNB(binarize = None)
gnb.fit(X_train, Y_train)

BernoulliNB(alpha=1.0, binarize=None, class_prior=None, fit_prior=True)

In [21]:
print("classifier score:", gnb.score(X_test, Y_test))
Y_pred = gnb.predict(X_test)
print("\naccuracy:",metrics.accuracy_score(Y_test, Y_pred))
print("\nConfusion matrix:")
print(metrics.confusion_matrix(Y_test, Y_pred))
print("\nPrecision Score per class:")
print(metrics.precision_score(Y_test, Y_pred, average = None))
print("\nAverage Precision Score:")
print(metrics.precision_score(Y_test, Y_pred, average = 'weighted'))
print("\nRecall Score per class:")
print(metrics.recall_score(Y_test, Y_pred, average= None))
print("\nAverage Recall Score:")
print(metrics.recall_score(Y_test, Y_pred, average = 'weighted'))
print("\nF1-score Score per class:")
print(metrics.f1_score(Y_test, Y_pred, average = None))
print("\nAverage F1 Score:")
print(metrics.f1_score(Y_test, Y_pred, average = 'weighted'))

report = metrics.classification_report(Y_test,Y_pred)
print(report)

classifier score: 0.9630148500980666

accuracy: 0.9630148500980666

Confusion matrix:
[[1893   34]
 [  98 1544]]

Precision Score per class:
[0.9507785  0.97845374]

Average Precision Score:
0.9635111277896611

Recall Score per class:
[0.98235599 0.94031669]

Average Recall Score:
0.9630148500980666

F1-score Score per class:
[0.96630934 0.95900621]

Average F1 Score:
0.9629493695236031
              precision    recall  f1-score   support

           0       0.95      0.98      0.97      1927
           1       0.98      0.94      0.96      1642

    accuracy                           0.96      3569
   macro avg       0.96      0.96      0.96      3569
weighted avg       0.96      0.96      0.96      3569



In [44]:
s = ['accuracy_weighted', 'precision_weighted', 'recall_weighted', 'f1_weighted']
scores_ = model_selection.cross_validate(lr_m, X, Y, scoring = s, cv = 5)                                 
print('****Logistic Regression mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_precision_weighted'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1 weighted:', scores_['test_f1_weighted'].mean())

****Logistic Regression mean scores****
scores: {'fit_time': array([2.18075132, 2.30030203, 2.24466872, 2.14507484, 2.04035211]), 'score_time': array([0.04765677, 0.04868722, 0.04739642, 0.04808712, 0.04802036]), 'test_accuracy': array([0.96813725, 0.97197898, 0.96882662, 0.96672504, 0.96882662]), 'test_average_precision': array([0.99414533, 0.99548947, 0.99406199, 0.99345015, 0.99433703]), 'test_recall_micro': array([0.96813725, 0.97197898, 0.96882662, 0.96672504, 0.96882662]), 'test_f1_micro': array([0.96813725, 0.97197898, 0.96882662, 0.96672504, 0.96882662])}

mean score test accuracy: 0.9688989045705847

mean score test precision weighted: 0.9942967924187471

mean score test recall weighted: 0.9688989045705847

mean score test f1 weighted: 0.9688989045705847


In [45]:
#sorted(sklearn.metrics.SCORERS.keys())